In [1]:
import json
from pathlib import Path
from bs4 import BeautifulSoup

In [2]:
data_dir = Path("../data")
content_files = []
for cat_dir in sorted(data_dir.iterdir()):
    content_dir = cat_dir / "content"
    if content_dir.exists():
        files = sorted(content_dir.glob("*.json"))
        content_files.extend(files)
        print(f"{cat_dir.name}: {len(files)} articles")
print(f"\nTotal: {len(content_files)} articles")

economy: 480 articles
politics: 480 articles
social: 480 articles

Total: 1440 articles


In [3]:
def extract_content_text(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")
    for img in soup.find_all("img"):
        img.decompose()
    return soup.get_text(separator="\n").strip()

def process_article(path: Path) -> dict:
    data = json.loads(path.read_text(encoding="utf-8"))
    return {
        "id": data["id"],
        "title": data["title"],
        "categoryName": data["categoryName"],
        "abstract": (data.get("abstract") or "").strip(),
        "publishedTime": data["publishTime"],
        "tags": [t["title"] for t in (data.get("tags") or [])],
        "content_text": extract_content_text(data.get("content") or ""),
    }

In [4]:
# Inspect a single article to verify extraction quality
sample_path = next(iter(content_files))
sample = process_article(sample_path)
print("ID:", sample["id"])
print("Title:", sample["title"])
print("Category:", sample["categoryName"])
print("Tags:", sample["tags"])
print("Abstract:", sample["abstract"][:120])
print("\n--- content_text (first 300 chars) ---")
print(sample["content_text"][:300])

ID: 356852
Title: สรุปราคาทองคำ 23 ก.ย.2568 ปิดตลาดวันนี้ บวก 1,000 บาท ผันผวน 14 ครั้ง
Category: เศรษฐกิจ
Tags: ['ราคาทองวันนี้', 'ราคาทองวันนี้ 1 สลึง ขาย', 'ราคาทองวันนี้ เยาวราช', 'ราคาทองวันนี้ 1 สลึงขาย', 'ราคาทองวันนี้เยาวราช', 'ราคาทองวันนี้ 1 สลึง', 'ราคาทอง', 'ราคาทองคำแท่ง', 'ราคาทองคำตลาดโลก', 'ราคาทองคำรูปพรรณ', 'ราคาทองรูปพรรณวันนี้', 'ราคาทองกรอบบ่าย', 'ราคาทองเยาวราช วันนี้', 'ราคาทองคำโลก', 'ราคาทองปี 2568', 'ข่าวราคาทองวันนี้', 'ข่าววันนี้', 'สรุปราคาทองคำวันนี้']
Abstract: ราคา ทองคำ ปิดตลาดผันผวน 14 ครั้ง บวกแรง 1,000 บาท “ทองรูปพรรณ” ในประเทศ ขายออกบาทละ 57,750  “ทองคำแท่ง” ขายออก 56, 950 

--- content_text (first 300 chars) ---
วันนี้ ( 23 ก.ย.2568) ราคาทองโลกเริ่มมีการปรับตัวขึ้น ทำ All Time High อีกครั้งในระดับ 3,788 ดอลลาร์ต่อออนซ์ในช่วงปิดตลาดเวลา 16.24 น. ท่ามกลางการจัดประชุมฉุกเฉิน UNSC โดยชาติพันธมิตรNATO ที่กล่าวหารัสเซียอย่างรุนแรงหลังละเมิดน่านฟ้าของเอสโตเนีย-โปแลนด์ โดยยูเครน เรียกร้องให้มีการตอบโต้อย่างแข็งกร้า


In [5]:
records = [process_article(f) for f in content_files]
print(f"Processed {len(records)} articles")

Processed 1440 articles


In [6]:
for r in records[:3]:
    print(f"[{r['id']}] {r['title']}")
    print(f"  tags: {r['tags'][:4]}")
    print(f"  content_text[:100]: {r['content_text'][:100]!r}")
    print()

[356852] สรุปราคาทองคำ 23 ก.ย.2568 ปิดตลาดวันนี้ บวก 1,000 บาท ผันผวน 14 ครั้ง
  tags: ['ราคาทองวันนี้', 'ราคาทองวันนี้ 1 สลึง ขาย', 'ราคาทองวันนี้ เยาวราช', 'ราคาทองวันนี้ 1 สลึงขาย']
  content_text[:100]: 'วันนี้ ( 23 ก.ย.2568) ราคาทองโลกเริ่มมีการปรับตัวขึ้น ทำ All Time High อีกครั้งในระดับ 3,788 ดอลลาร์'

[356854] เช็คให้ชัวร์ ลดละเมิดลิขสิทธิ์ กรมทรัพย์สินฯเปิดระบบให้บริการฐานข้อมูลเพลง
  tags: ['กรมทรัพย์สินทางปัญญา', 'ละเมิดทรัพย์สินทางปัญญา', 'ละเมิดลิขสิทธิ์', 'ละเมิดลิขสิทธิ์เพลง']
  content_text[:100]: 'วันนี้ (23 ก.ย.2568) น.ส.นุสรา กาญจนกูล อธิบดีกรมทรัพย์สินทางปัญญา เปิดเผยว่า กรมให้ความสำคัญในการส่'

[356859] ททท.ดึงจีนเที่ยวไทย รับหยุดยาว พร้อมเปิดตัว Nihao Month ปีที่ 2
  tags: ['นักท่องเที่ยวจีน', 'ตลาดจีน', 'ฉลองวันชาติจีน', 'วันชาติจีน']
  content_text[:100]: 'วันนี้ ( 23 ก.ย.2568) นางสาวฐาปนีย์ เกียรติไพบูลย์ ผู้ว่าการการท่องเที่ยวแห่งประเทศไทย (ททท.)กล่าวว่'



In [7]:
out_path = data_dir / "dataset.jsonl"
with out_path.open("w", encoding="utf-8") as f:
    for record in records:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")
print(f"Written {len(records)} records \u2192 {out_path}")

Written 1440 records → ../data/dataset.jsonl


In [8]:
import pandas as pd

df = pd.DataFrame(records)
print(df.shape)

df.head()

(1440, 7)


,id,title,categoryName,abstract,publishedTime,tags,content_text
0,356852,"สรุปราคาทองคำ 23 ก.ย.2568 ปิดตลาดวันนี้ บวก 1,...",เศรษฐกิจ,"ราคา ทองคำ ปิดตลาดผันผวน 14 ครั้ง บวกแรง 1,000...",2025-09-23T17:01:11+07:00,"[ราคาทองวันนี้, ราคาทองวันนี้ 1 สลึง ขาย, ราคา...",วันนี้ ( 23 ก.ย.2568) ราคาทองโลกเริ่มมีการปรับ...
1,356854,เช็คให้ชัวร์ ลดละเมิดลิขสิทธิ์ กรมทรัพย์สินฯเป...,เศรษฐกิจ,กรมทรัพย์สินฯให้บริการระบบฐานข้อมูลเพลงไทยกว่า...,2025-09-23T17:30:00+07:00,"[กรมทรัพย์สินทางปัญญา, ละเมิดทรัพย์สินทางปัญญา...",วันนี้ (23 ก.ย.2568) น.ส.นุสรา กาญจนกูล อธิบดี...
2,356859,ททท.ดึงจีนเที่ยวไทย รับหยุดยาว พร้อมเปิดตัว Ni...,เศรษฐกิจ,ททท.เดินเครื่องเร่งดึงนักท่องเที่ยวจีนกลับไทย ...,2025-09-23T18:10:00+07:00,"[นักท่องเที่ยวจีน, ตลาดจีน, ฉลองวันชาติจีน, วั...",วันนี้ ( 23 ก.ย.2568) นางสาวฐาปนีย์ เกียรติไพบ...
3,356889,ภาษีสหรัฐฯ-ราคาสินค้าเกษตร ฉุดส่งออกไทยขยายตั...,เศรษฐกิจ,สนค. เผยส่งออกไทยส.ค.68 ขยายตัว 5.8% ต่อเนื่อง...,2025-09-24T15:35:30+07:00,"[ส่งออก, ส่งออกข้าว2568, ส่งออกไทย, สินค้าส่งอ...",วันนี้ ( 24 ก.ย.2568) นายพูนพงษ์ นัยนาภากรณ์ ผ...
4,356895,สรุปราคาทองคำ 24 ก.ย.2568 ปิดตลาดวันนี้ บวก 50...,เศรษฐกิจ,ราคา ทองคำ ปิดตลาดผันผวน 8 ครั้ง บวก 50 บาท “ท...,2025-09-24T16:53:48+07:00,"[ราคาทองวันนี้, ราคาทอง, ราคาทองวันนี้ 1 สลึง ...",วันนี้ ( 24 ก.ย.2568) ราคาทองโลกปรับตัวขึ้นต่อ...


In [9]:
df.describe()

,id,title,categoryName,abstract,publishedTime,tags,content_text
count,1440,1440,1440,1440,1440,1440,1440
unique,1440,1440,3,1281,1431,1376,1440
top,502104,สธ.ฉีดวัคซีนฟรีชาวไทยมุสลิม เบ็ดเสร็จใน 1 วัน ...,เศรษฐกิจ,,2026-01-21T09:00:00+07:00,"[ราคาทองวันนี้, ราคาทอง, ราคาทองคำแท่ง, ราคาทอ...",วันนี้ (11 ก.พ.2569) ที่ศูนย์บริหารกิจการศาสนา...
freq,1,1,480,160,3,28,1


In [10]:
df.groupby("categoryName").describe()

id                     title         \
             count unique     top freq count unique   
categoryName                                          
การเมือง       480    480  502211    1   480    480   
สังคม          480    480  502104    1   480    480   
เศรษฐกิจ       480    480  501829    1   480    480   

                                                                     abstract  \
                                                            top freq    count   
categoryName                                                                    
การเมือง      ผลสำรวจ "นิด้าโพล" ตัวอย่าง 58.28% ชี้ กกต.ลงโ...    1      480   
สังคม         สธ.ฉีดวัคซีนฟรีชาวไทยมุสลิม เบ็ดเสร็จใน 1 วัน ...    1      480   
เศรษฐกิจ      เงินเฟ้อติดลบ 10 เดือน คาดไตรมาส 1 ลบ 0.43 % ส...    1      480   

                     ...              publishedTime       tags         \
             unique  ...                        top freq count unique   
categoryName         ...                                                
การเมือง        400  ...  2026-01-21T09:00:00+07:00    3   480    468   
สังคม           442  ...  2025-10-27T08:06:00+07:00    2   480    460   
เศรษฐกิจ        441  ...  2026-01-21T10:38:48+07:00    2   480    448   

                                                                      \
                                                            top freq   
categoryName                                                           
การเมือง      [ผลเลือกตั้ง69, เลือกตั้ง, เลือกตั้ง2569, เลือ...    3   
สังคม         [กรมอุตุนิยมวิทยา, ฝนตก, ฤดูหนาว, อากาศวันนี้,...   18   
เศรษฐกิจ      [ราคาทองวันนี้, ราคาทอง, ราคาทองคำแท่ง, ราคาทอ...   28   

             content_text         \
                    count unique   
categoryName                       
การเมือง              480    480   
สังคม                 480    480   
เศรษฐกิจ              480    480   

                                                                      
                                                            top freq  
categoryName                                                          
การเมือง      วันนี้ (15 ก.พ.2569) ศูนย์สำรวจความคิดเห็น "นิ...    1  
สังคม         วันนี้ (11 ก.พ.2569) ที่ศูนย์บริหารกิจการศาสนา...    1  
เศรษฐกิจ      วันนี้ (5 ก.พ.2569) น.ส.ณัฐิยา สุจินดา รองผู้อ...    1  

[3 rows x 24 columns]